## VAR vs VARMAX Model Validation


In this notebook, we will validate which model performs better given computational constraints: a VAR model retrained after each prediction or a VARMAX retrained every 20 predictions. To do so, we will retrun the model and functions constructed in [Systems_Identification_Fitting](Systems_Identification_Fitting.ipynb).

In [159]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats
import math as m
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, VARMAX
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 


states = pd.read_csv('states.csv')
del states['Unnamed: 0']
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM)
0,0.000858,3.051840,12388143.0,0.0,2.632464e+07,1.157921e+32,230002.364951,583.0,0.978704,0.999998,0.99998,3.005443,11977.048864,1.389236e+07,2.601571e+07,1249.4226,293469.366076,3521.749280
1,0.000858,3.051840,12387897.0,0.0,2.632464e+07,1.157921e+32,230002.364951,583.0,0.978704,0.999998,0.99998,3.005450,12025.302340,1.383645e+07,2.601571e+07,1249.4226,293469.366076,3484.967617
2,0.000869,3.060763,12387633.0,0.0,2.629462e+07,1.157921e+32,229927.716817,583.0,0.978704,0.999998,0.99998,3.005465,12033.680708,1.382679e+07,2.598571e+07,1249.4226,293469.366076,3509.940000
3,0.000870,3.051668,12387345.0,0.0,2.629462e+07,1.157921e+32,229927.716817,583.0,0.978704,0.999998,0.99998,3.005465,12033.680708,1.382679e+07,2.598571e+07,1249.4226,293469.366076,3438.463957
4,0.000875,3.043224,12387095.0,0.0,2.629407e+07,1.157921e+32,229903.041060,583.0,0.978704,0.999998,0.99998,3.005472,12080.165571,1.377424e+07,2.598521e+07,1249.4226,293469.366076,3464.125574


In [160]:
# add additional state variables
states['RedemptionPriceinEth'] = states['RedemptionPrice'] / states['ETH Price (OSM)']
states['RedemptionPriceError'] = states['RedemptionPrice'] - states['marketPriceUsd']

In [161]:
# define constants (will come from cadCAD model but added here for calculations)
params = {
    'liquidation_ratio': 1.45,
    'debt_ceiling': 1e9,
    'uniswap_fee': 0.003,
    'arbitrageur_considers_liquidation_ratio': True,
}


## Create Arbtrageur data vector $u^*$

In [162]:
def get_aggregated_arbitrageur_decision(params, state):

    # This Boolean indicates whether or not the arbitrageur is rationally considering 
    # borrowing to the liquidation ratio limit. If TRUE, arbitrage opportunities are less
    # frequent when RAI is expensive and more frequent when RAI is cheap. If FALSE, only 
    # the difference in market and redemption prices (net of Uniswap fee) matters for trading, 
    # which may conform more to individual trader expectations and behavior.
    consider_liquidation_ratio = params['arbitrageur_considers_liquidation_ratio']
    
    # These are the states of the SAFE balances in aggregate & its fixed parameters
    total_borrowed = state['SAFE_Debt'] # D
    total_collateral = state['SAFE_Collateral'] # Q
    liquidation_ratio = params['liquidation_ratio']
    debt_ceiling = params['debt_ceiling']


    # These are the states of the Uniswap secondary market balances and its fee
    RAI_balance = state['RAI_balance'] # R_Rai
    ETH_balance = state['ETH_balance'] # R_Eth
    uniswap_fee = params['uniswap_fee']

    # These are the prices of RAI in USD/RAI for SAFE redemption and the market price oracle, resp.
    redemption_price = state['target_price'] # $p^r_{U/R}
    market_price = state['market_price'] # p_{U/R} > 0
    
    # This is the price of ETH in USD/ETH
    eth_price = state['eth_price'] # p_{U/E}

        
    # These functions define the optimal borrowing/repayment decisions of the aggregated arbitrageur
    def g1(RAI_balance, ETH_balance, uniswap_fee, liquidation_ratio, redemption_price):
        return ((eth_price * RAI_balance * ETH_balance * (1 - uniswap_fee)) / (liquidation_ratio * redemption_price)) ** 0.5

    def g2(RAI_balance, ETH_balance, uniswap_fee, liquidation_ratio, redemption_price):
        return (RAI_balance * ETH_balance * (1 - uniswap_fee) * liquidation_ratio * (redemption_price / eth_price)) ** 0.5

    # This Boolean resolves to TRUE if the agg. arb. acts this timestep when RAI is expensive 
    # on the secondary market
    expensive_RAI_on_secondary_market = \
        redemption_price < ((1 - uniswap_fee) / liquidation_ratio) * market_price  \
        if consider_liquidation_ratio \
        else redemption_price < (1 - uniswap_fee) * market_price
        
    # This Boolean resolves to TRUE if the agg. arb. acts this timestep when RAI is cheap 
    # on the secondary market
    cheap_RAI_on_secondary_market = \
        redemption_price > (1 / ((1 - uniswap_fee) * liquidation_ratio)) * market_price  \
        if consider_liquidation_ratio \
        else redemption_price > (1 / (1 - uniswap_fee)) * market_price

    if expensive_RAI_on_secondary_market:
        '''
        Expensive RAI on Uni:
        (put ETH from pocket into additional collateral in SAFE)
        draw RAI from SAFE -> Uni
        ETH from Uni -> into pocket
        '''
        _g1 = g1(RAI_balance, ETH_balance, uniswap_fee,
                 liquidation_ratio, redemption_price)
        d = (_g1 - RAI_balance) / (1 - uniswap_fee) # should be \geq 0
        q = ((liquidation_ratio * redemption_price) /
                     eth_price) * (total_borrowed + d) - total_collateral # should be \geq 0
        z = -(ETH_balance * d * (1 - uniswap_fee)) / \
            (RAI_balance + d * (1 - uniswap_fee)) # should be leq 0
        r = d # should be \geq 0
        
    elif cheap_RAI_on_secondary_market:
        '''
        Cheap RAI on Uni:
        ETH out of pocket -> Uni
        RAI from UNI -> SAFE to wipe debt
        (and collect collateral ETH from SAFE into pocket)
        '''

        _g2 = g2(RAI_balance, ETH_balance, uniswap_fee,
                 liquidation_ratio, redemption_price)
        z = (_g2 - ETH_balance) / (1 - uniswap_fee) # should be \geq 0
        r = -(RAI_balance * z * (1 - uniswap_fee)) / \
            (ETH_balance + z * (1 - uniswap_fee)) # should be \leq 0
        d = r # should be \leq 0
        q = ((liquidation_ratio * redemption_price /
             eth_price) * (total_borrowed + d) - total_collateral) # should be \leq 0
    else:
        pass

    return {
        'q' : q,
        'd' : d,
        'r' : r,
        'z' : z
    }

In [163]:
# subset state variables for arbitrageur vector
state_subset = states[['marketPriceUsd','RedemptionPrice','ETH Price (OSM)','collateral',
                       'EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]
# map state data to arbitrageur vector fields 
state_subset.columns = ['market_price','target_price','eth_price','SAFE_Collateral',
                       'ETH_balance','RAI_balance','SAFE_Debt']

In [164]:
# create list of u^* vectors
values = []

# iterate through real data to create u^* and save to values
for i in range(0,len(state_subset)):
    values.append(get_aggregated_arbitrageur_decision(params,state_subset.loc[i]))
    
# create historic u^* dataframe
local = pd.DataFrame(values)
local.columns = ['Q','D','Rrai','Reth']
local.head()

,Q,D,Rrai,Reth
0,28122.869039,-2.279045e+06,-2.279045e+06,2357.497660
1,28426.718797,-2.284020e+06,-2.284020e+06,2384.664437
2,28241.725552,-2.233082e+06,-2.233082e+06,2324.799929
3,28704.381800,-2.351735e+06,-2.351735e+06,2473.643064
4,28602.135401,-2.256182e+06,-2.256182e+06,2373.409556


In [165]:
# function to create coordinate transformations
def coordinate_transformations(params,df,Q,R_eth,R_rai,D,RedemptionPrice,EthPrice):
    '''
    Description:
    Function that takes in pandas dataframe and the names of columns
    
    Parameters:
    df: pandas dataframe containing states information
    Q: dataframe column name
    R_eth: dataframe column name
    R_rai: dataframe column name
    D: dataframe column name
    RedemptionPrice: dataframe column name
    EthPrice: dataframe column name

    Returns: Pandas dataframe with alpha, beta, gamma, delta transformed values
    
    Example:
    
    coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]
    '''
    
    # Calculate alpha
    d = df[D].diff()
    d.fillna(0,inplace=True)
    df['d'] = d
    
    df['alpha'] = df['d'] / params['debt_ceiling']

    # calculate beta
    df['C_o'] = (df[RedemptionPrice]/states[EthPrice]) * params['liquidation_ratio']

    q = df[Q].diff()
    q.fillna(0,inplace=True)
    df['q'] = q


    df['C_1'] = (df['C_o'] * df[D]) - df[Q]

    df['beta'] = (df['q'] - (df['C_o']*df['d']))/ df['C_1']

    # calculate gamma
    r = df[R_rai].diff()
    r.fillna(0,inplace=True)
    df['r'] = r

    df['gamma'] = df['r']/df[R_rai]

    # calculate delta
    z = df[R_eth].diff()
    z.fillna(0,inplace=True)
    df['z'] = z

    df['delta'] = df['z']/df[R_eth]
    
    return df

In [166]:
transformed = coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed

,alpha,beta,gamma,delta
0,0.000000e+00,0.000000,0.000000,0.000000
1,0.000000e+00,0.000000,-0.004041,0.004013
2,-3.000000e-05,0.001201,-0.000699,0.000696
3,0.000000e+00,0.000000,0.000000,0.000000
4,-5.000000e-07,0.000020,-0.003815,0.003848
...,...,...,...,...
1942,-1.646075e-06,0.108212,0.249397,-0.333334
1943,0.000000e+00,-0.000000,0.000000,0.000000
1944,0.000000e+00,-0.000000,0.000000,0.000000
1945,0.000000e+00,-0.000000,0.000000,0.000000


In [167]:
# add additional signals to arbitrageur state
local['RedemptionPrice'] = states['RedemptionPrice']
local['ETH Price (OSM)'] = states['ETH Price (OSM)']

local

,Q,D,Rrai,Reth,RedemptionPrice,ETH Price (OSM)
0,28122.869039,-2.279045e+06,-2.279045e+06,2357.497660,3.005443,3521.749280
1,28426.718797,-2.284020e+06,-2.284020e+06,2384.664437,3.005450,3484.967617
2,28241.725552,-2.233082e+06,-2.233082e+06,2324.799929,3.005465,3509.940000
3,28704.381800,-2.351735e+06,-2.351735e+06,2473.643064,3.005465,3438.463957
4,28602.135401,-2.256182e+06,-2.256182e+06,2373.409556,3.005472,3464.125574
...,...,...,...,...,...,...
1942,-9.603730,-1.254849e+02,-1.254849e+02,0.263944,3.140000,1825.887144
1943,-9.488397,-1.314605e+02,-1.314605e+02,0.278977,3.140000,1793.770282
1944,-9.525170,-1.295717e+02,-1.295717e+02,0.274197,3.140000,1803.891149
1945,-9.525170,-1.295717e+02,-1.295717e+02,0.274197,3.140000,1803.891149


In [168]:
transformed_arbitrageur = coordinate_transformations(params,local,'Q','Reth','Rrai',
                           'D','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed_arbitrageur

,alpha,beta,gamma,delta
0,0.000000e+00,-0.000000,-0.000000,0.000000
1,-4.974383e-06,-0.009912,0.002178,0.011392
2,5.093809e-05,0.008004,-0.022811,-0.025750
3,-1.186537e-04,-0.019348,0.050454,0.060172
4,9.555367e-05,0.007075,-0.042352,-0.042232
...,...,...,...,...
1942,-1.977733e-07,0.105609,1.576073,1.766264
1943,-5.975671e-09,0.014255,0.045456,0.053888
1944,1.888822e-09,-0.004516,-0.014577,-0.017436
1945,0.000000e+00,0.000000,-0.000000,0.000000


In [169]:
def create_transformed_errors(transformed_states,transformed_arbitrageur):
    '''
    Description:
    Function for taking two pandas dataframes of transformed states and taking the difference
    to produce an error dataframe. 
    
    Parameters:
    transformed_states: pandas dataframe with alpha, beta, gamma, and delta features
    transformed_arbitrageur: pandas dataframe with alpha, beta, gamma, and delta features

    Returns:
    error pandas dataframe and transformation object
    
    '''
    alpha_diff = transformed_states['alpha'] - transformed_arbitrageur['alpha']
    beta_diff = transformed_states['beta'] - transformed_arbitrageur['beta']
    gamma_diff = transformed_states['gamma'] - transformed_arbitrageur['gamma']
    delta_diff = transformed_states['delta'] - transformed_arbitrageur['delta']


    e_u = pd.DataFrame(alpha_diff)
    e_u['beta'] = beta_diff
    e_u['gamma'] = gamma_diff
    e_u['delta'] = delta_diff

    e_u = e_u.astype(float)
    
    return e_u

e_u = create_transformed_errors(transformed,transformed_arbitrageur)
e_u.head()

,alpha,beta,gamma,delta
0,0.000000,0.000000,0.000000,0.000000
1,0.000005,0.009912,-0.006219,-0.007380
2,-0.000081,-0.006803,0.022112,0.026447
3,0.000119,0.019348,-0.050454,-0.060172
4,-0.000096,-0.007055,0.038537,0.046080


In [170]:
def power_transformation(e_u):
    '''
    Definition:
    Function to perform a power transformation on the coordinate 
    transformed differenced data
    
    Parameters:
    e_u: Dataframe of coordinated transformed differenced data
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    Transformed dataframe and transformation object
    
    Example:
    transformed_df, pt = power_transformation(e_u)
    '''
    pt = PowerTransformer()
    yeo= pd.DataFrame(pt.fit_transform(e_u),columns=e_u.columns)

    return yeo, pt

In [171]:
e_u_yeo, pt = power_transformation(e_u)

<!-- ### Autogressive lag selection -->

In [172]:
# split data between train and test (in production deployment, can remove)
split_point = int(len(e_u) * .8)
train = e_u_yeo.iloc[0:split_point]
test = e_u_yeo.iloc[split_point:]


states_train = states.iloc[0:split_point]
states_test = states.iloc[split_point:]

In [173]:
def VARMAX_prediction(e_u,RedemptionPriceError,newRedemptionPriceError,steps=1,lag=1):
    '''
    Description:
    Function to train and forecast a VARMAX model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    RedemptionPriceErrorPrevious: 1d Numpy array of RedemptionPriceError values
    newRedemptionPriceError: exogenous latest redemption price error signal - float
    steps: Number of forecast steps. Default is 1
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    Y_pred = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'][0:5],steps=5,lag=1)
    '''
    # instantiate the VARMAX model object from statsmodels 
    model = VARMAX(endog=e_u.values,exog=RedemptionPriceError,
                   initialization='approximate_diffuse',measurement_error=True)

    # fit model with determined lag values
    results = model.fit(order=(lag,0))
    
    Y_pred = results.forecast(steps = steps, exog=newRedemptionPriceError)
    
    return Y_pred.values

In [174]:
def VAR_prediction(e_u,lag=1):
    '''
    Description:
    Function to train and forecast a VAR model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    VAR_prediction(e_u,6)    
    '''
    # instantiate the VAR model object from statsmodels 
    model = VAR(e_u)

    # fit model with determined lag values
    results = model.fit(lag)
    
    lag_order = results.k_ar
    
    Y_pred = results.forecast(e_u.values[-lag_order:],1)

    
    return Y_pred[0]

In [175]:
varmax_predictions = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'].values[0:20],steps=20,lag=1)

In [176]:
var_predictions = []
train_var = train.copy()
for i in range(0,20): 
    var_predictions.append(VAR_prediction(train_var,6))
    train_var = train_var.append(test.iloc[i])

In [177]:
def invert_power_transformation(pt,prediction):
    '''
    Definition:
    Function to invert power transformation
    
    Parameters:
    pt: transformation object
    prediction: Numpy array of model state coordinate transformed percentage changes
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    inverted transformation numpy array
    
    Example:
    inverted_array = invert_power_transformation(pt,prediction)
    
    '''
    # transform back into coordinate system
    inverted = pt.inverse_transform(prediction)
    
    return inverted

In [178]:
var_predictions = invert_power_transformation(pt,var_predictions)
varmax_predictions = invert_power_transformation(pt,varmax_predictions)

## Model Evaluation Comparision (in coordinate transformed and power transformed differenced space)  

In [179]:
var_predictions_df = pd.DataFrame(var_predictions,columns=['alpha','beta','gamma','delta'])
varmax_predictions_df = pd.DataFrame(varmax_predictions,columns=['alpha','beta','gamma','delta'])

## Alpha - Root Mean Square Error

In [180]:
var_alpha_rmse = m.sqrt(mean_squared_error(var_predictions_df['alpha'], test.head(20)['alpha']))
varmax_alpha_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['alpha'], test.head(20)['alpha']))
if var_alpha_rmse >= varmax_alpha_rmse:
    print('VARMAX performs better by {}'.format(varmax_alpha_rmse-var_alpha_rmse))
else:
    print('VAR performs better by {}'.format(var_alpha_rmse-varmax_alpha_rmse))

VAR performs better by -3.1612026293759143e-05


## Beta - Root Mean Square Error

In [181]:
var_beta_rmse = m.sqrt(mean_squared_error(var_predictions_df['beta'], test.head(20)['beta']))
varmax_beta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['beta'], test.head(20)['beta']))
if var_beta_rmse >= varmax_beta_rmse:
    print('VARMAX performs better by {}'.format(varmax_beta_rmse-var_beta_rmse))
else:
    print('VAR performs better by {}'.format(var_beta_rmse-varmax_beta_rmse))

VARMAX performs better by -0.001590044088887188


## Gamma - Root Mean Square Error

In [182]:
var_gamma_rmse = m.sqrt(mean_squared_error(var_predictions_df['gamma'], test.head(20)['gamma']))
varmax_gamma_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['gamma'], test.head(20)['gamma']))
if var_gamma_rmse >= varmax_gamma_rmse:
    print('VARMAX performs better by {}'.format(varmax_gamma_rmse-var_gamma_rmse))
else:
    print('VAR performs better by {}'.format(var_gamma_rmse-varmax_gamma_rmse))

VARMAX performs better by -0.007534888065225429


## Delta - Root Mean Square Error

In [183]:
var_delta_rmse = m.sqrt(mean_squared_error(var_predictions_df['delta'], test.head(20)['delta']))
varmax_delta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['delta'], test.head(20)['delta']))
if var_delta_rmse >= varmax_delta_rmse:
    print('VARMAX performs better by {}'.format(varmax_delta_rmse-var_delta_rmse))
else:
    print('VAR performs better by {}'.format(var_delta_rmse-varmax_delta_rmse))

VARMAX performs better by -0.007941736103854243


## Model Evaluation Comparision as States

In [184]:
def inverse_transformation_and_state_update(Y_pred,previous_state,params):
    '''
    Description:
    Function to take system identification model prediction and invert transfrom and create new state
    
    Parameters:
    y_pred: numpy array of transformed state changes
    previous_state: pandas dataframe of previous state or 'current' state
    params: dictionary of system parameters
    
    Returns:
    pandas dataframe of new states 
    
    Example:
    inverse_transformation_and_state_update(Y_pred,previous_state,params)
    '''
    
    d_star = Y_pred[0] * params['debt_ceiling']
    
    q_star = previous_state['C_o'] * params['debt_ceiling'] * Y_pred[0] + previous_state['C_1'] * Y_pred[1]
    
    r_star = Y_pred[2] * previous_state['gamma'] * previous_state['RaiInUniswap']

    z_star = Y_pred[3] * previous_state['delta'] * previous_state['EthInUniswap']

    new_state = pd.DataFrame(previous_state[['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']].to_dict())
    new_state['Q'] = new_state['collateral'] + q_star
    new_state['D'] = new_state['RaiDrawnFromSAFEs'] + d_star
    new_state['R_Rai'] = new_state['RaiInUniswap'] + r_star
    new_state['R_Eth'] = new_state['EthInUniswap'] + z_star
    
    return new_state[['Q','D','R_Rai','R_Eth']]



In [185]:
VAR_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VAR_new_states.append(inverse_transformation_and_state_update(var_predictions[i],previous_state,params))
    index += 1
    
VAR_new_states = pd.concat(VAR_new_states)
VAR_new_states

,Q,D,R_Rai,R_Eth
1556,7755.851392,3.405465e+07,2.971660e+07,61652.901778
1557,7905.440842,3.385084e+07,2.962366e+07,61699.044956
1558,5735.137218,3.424498e+07,3.020418e+07,60719.239840
1559,5367.192894,3.389079e+07,2.997530e+07,60996.162303
1560,6067.236767,3.413323e+07,2.998934e+07,60974.687365
1561,5776.592667,3.403075e+07,3.002632e+07,60900.567144
1562,4754.134425,3.376733e+07,2.977170e+07,60653.871512
1563,6826.076896,3.405493e+07,2.953919e+07,59903.729943
1564,6139.223102,3.343107e+07,2.943983e+07,60268.187346
1565,6700.253445,3.345664e+07,2.922273e+07,60089.045961


In [186]:
VARMAX_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VARMAX_new_states.append(inverse_transformation_and_state_update(varmax_predictions[i],previous_state,params))
    index += 1
    
VARMAX_new_states = pd.concat(VARMAX_new_states)
VARMAX_new_states

,Q,D,R_Rai,R_Eth
1556,7134.500940,3.379992e+07,2.971784e+07,61656.119961
1557,6423.776667,3.357798e+07,2.961970e+07,61700.918608
1558,6489.464814,3.419653e+07,3.018731e+07,60752.266224
1559,6392.337423,3.402582e+07,2.998008e+07,60989.541656
1560,6410.324475,3.406835e+07,2.998924e+07,60974.878517
1561,6352.099465,3.407088e+07,3.002658e+07,60899.945970
1562,6364.509959,3.389021e+07,2.978054e+07,60663.984366
1563,6558.815433,3.372880e+07,2.954197e+07,59913.948038
1564,6390.432350,3.345430e+07,2.943946e+07,60269.631113
1565,6422.223439,3.330032e+07,2.922155e+07,60087.889602


In [187]:
test_data = states.iloc[test.index[0:20]][['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]


In [188]:
test_data

,collateral,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs
1557,6112.1732,61701.008091,2.961953e+07,3.369223e+07
1558,6112.1732,60746.845297,3.019005e+07,3.427256e+07
1559,6112.1732,60990.371859,2.997948e+07,3.409167e+07
1560,6112.1732,60974.811993,2.998928e+07,3.412678e+07
1561,6112.1732,60899.716831,3.002668e+07,3.412678e+07
1562,6112.1732,60663.156241,2.977982e+07,3.394178e+07
1563,6112.1732,59907.912377,2.954037e+07,3.379492e+07
1564,6112.1732,60270.860599,2.943917e+07,3.352893e+07
1565,6112.1732,60087.012619,2.922068e+07,3.336193e+07
1566,6112.1732,60163.059766,2.919058e+07,3.336193e+07


## Q - Root Mean Square Error

In [189]:
var_q_rmse = m.sqrt(mean_squared_error(VAR_new_states['Q'], test_data['collateral']))
varmax_q_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['Q'], test_data['collateral']))
if var_q_rmse >= varmax_q_rmse:
    print('VARMAX performs better by {}'.format(varmax_q_rmse-var_q_rmse))
else:
    print('VAR performs better by {}'.format(var_q_rmse-varmax_q_rmse))

VARMAX performs better by -358.1284637885369


## D - Root Mean Square Error

In [190]:
var_d_rmse = m.sqrt(mean_squared_error(VAR_new_states['D'], test_data['RaiDrawnFromSAFEs']))
varmax_d_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['D'], test_data['RaiDrawnFromSAFEs']))
if var_d_rmse >= varmax_d_rmse:
    print('VARMAX performs better by {}'.format(varmax_d_rmse-var_d_rmse))
else:
    print('VAR performs better by {}'.format(var_d_rmse-varmax_d_rmse))

VARMAX performs better by -19520.10687361713


## R_Rai - Root Mean Square Error

In [191]:
var_rai_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Rai'], test_data['RaiInUniswap']))
varmax_rai_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Rai'], test_data['RaiInUniswap']))
if var_rai_rmse >= varmax_rai_rmse:
    print('VARMAX performs better by {}'.format(varmax_rai_rmse-var_rai_rmse))
else:
    print('VAR performs better by {}'.format(var_rai_rmse-varmax_rai_rmse))

VAR performs better by -574.0467929494625


## R_Eth - Root Mean Square Error

In [192]:
var_eth_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Eth'], test_data['EthInUniswap']))
varmax_eth_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Eth'], test_data['EthInUniswap']))
if var_eth_rmse >= varmax_eth_rmse:
    print('VARMAX performs better by {}'.format(varmax_eth_rmse-var_eth_rmse))
else:
    print('VAR performs better by {}'.format(var_eth_rmse-varmax_eth_rmse))

VARMAX performs better by -0.34020523839387806


## Conclusion


Although only trained every 20 timesteps, VARMAX, with the addition of an 